In [11]:
import os
import pandas as pd

# Remit

In [12]:
# Ordnerpfad zu den CSV-Dateien
csv_folder = 'data/Json_Dateien'

# Leere Liste zum Speichern der DataFrames
dataframes = []

# Alle CSV-Dateien im Ordner einlesen
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        csv_file = os.path.join(csv_folder, filename)  # Vollständiger Pfad zur CSV-Datei
        df = pd.read_csv(csv_file)  # Einlesen der CSV-Datei
        dataframes.append(df)  # Hinzufügen des DataFrames zur Liste

# Alle DataFrames zu einem einzigen DataFrame zusammenführen
df_remit = pd.concat(dataframes, ignore_index=True)

In [13]:
df_remit['publishTime'] = pd.to_datetime(df_remit['publishTime'])
df_remit['createdTime'] = pd.to_datetime(df_remit['createdTime'])

# Versuche, 'eventStatus' und 'eventStartTime' in Datetime-Objekte zu konvertieren; ungültige Werte werden in NaT umgewandelt
df_remit['eventStatus'] = pd.to_datetime(df_remit['eventStatus'], errors='coerce')
df_remit['eventStartTime'] = pd.to_datetime(df_remit['eventStartTime'], errors='coerce')
df_remit['eventEndTime'] = pd.to_datetime(df_remit['eventEndTime'], errors='coerce')


df_remit['eventTime'] = df_remit['eventEndTime'] - df_remit['eventStartTime']

C:\Users\Michael Jäckle\AppData\Local\Temp\ipykernel_18260\998120984.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_remit['eventStatus'] = pd.to_datetime(df_remit['eventStatus'], errors='coerce')


In [14]:
df_remit = df_remit.filter(['publishTime','assetId','cause','eventTime'])
df_remit

,publishTime,assetId,cause,eventTime
0,2020-11-26 13:54:00+00:00,T_HOWAO-2,Planned Outage,0 days 09:00:00
1,2020-12-10 16:12:00+00:00,T_HOWAO-2,Planned Outage,0 days 10:00:00
2,2020-12-13 19:13:00+00:00,T_HOWAO-2,Planned Outage,0 days 02:00:00
3,2020-12-07 21:19:00+00:00,T_HOWAO-2,Planned Outage,0 days 10:00:00
4,2020-12-14 09:29:00+00:00,T_HOWAO-2,Planned Outage,0 days 03:00:00
...,...,...,...,...
431,2024-09-19 08:33:30+00:00,T_HOWAO-1,Under Investigation,1 days 23:46:00
432,2024-09-20 11:51:24+00:00,T_HOWAO-1,Under Investigation,1 days 21:46:00
433,2024-09-26 09:57:50+00:00,T_HOWAO-1,Planned Outage,6 days 04:00:00
434,2024-09-29 16:45:46+00:00,T_HOWAO-1,Under Investigation,1 days 00:00:00


In [15]:
# Funktion zum Runden auf das nächste 30-Minuten-Intervall
def round_to_nearest_half_hour(dt):
    if dt.minute < 15:
        return dt.replace(minute=0, second=0)
    elif dt.minute < 45:
        return dt.replace(minute=30, second=0)
    else:
        return dt.replace(minute=0, second=0) + pd.Timedelta(hours=1)

# Funktion zum Runden auf die nächste volle Stunde
def round_to_nearest_hour(dt):
    if dt.minute < 30:
        return dt.replace(minute=0, second=0)
    else:
        return dt.replace(minute=0, second=0) + pd.Timedelta(hours=1)

# Anwendung der Rundungsfunktionen auf df_tt
df_remit['rounded_half_hour'] = df_remit['publishTime'].apply(round_to_nearest_half_hour)

df_remit = df_remit.filter(['assetId',	'cause'	,'eventTime',	'rounded_half_hour'	])

In [16]:
df_remit

,assetId,cause,eventTime,rounded_half_hour
0,T_HOWAO-2,Planned Outage,0 days 09:00:00,2020-11-26 14:00:00+00:00
1,T_HOWAO-2,Planned Outage,0 days 10:00:00,2020-12-10 16:00:00+00:00
2,T_HOWAO-2,Planned Outage,0 days 02:00:00,2020-12-13 19:00:00+00:00
3,T_HOWAO-2,Planned Outage,0 days 10:00:00,2020-12-07 21:30:00+00:00
4,T_HOWAO-2,Planned Outage,0 days 03:00:00,2020-12-14 09:30:00+00:00
...,...,...,...,...
431,T_HOWAO-1,Under Investigation,1 days 23:46:00,2024-09-19 08:30:00+00:00
432,T_HOWAO-1,Under Investigation,1 days 21:46:00,2024-09-20 12:00:00+00:00
433,T_HOWAO-1,Planned Outage,6 days 04:00:00,2024-09-26 10:00:00+00:00
434,T_HOWAO-1,Under Investigation,1 days 00:00:00,2024-09-29 17:00:00+00:00
